In [ ]:
# TIDAK PERLU DIJALANKAN
# !wget https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Training_Input.zip
# !wget https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Training_Metadata.csv
# !wget https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Training_GroundTruth.csv
# !wget https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Test_Input.zip
# !unzip drive/My\ Drive/FP_ISIC_Melanoma_Dataset/ISIC_2019_Test_Input.zip -d drive/My\ Drive/FP_ISIC_Melanoma_Dataset

In [ ]:
# Indra
# !unzip ISIC_2019_Test_Input.zip -d FP_ISIC_Melanoma_Dataset
# !pip3 install pydicom

In [3]:
import pandas as pd
import os
from shutil import copyfile
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from os import getcwd
import shutil
from tensorflow.keras.optimizers import RMSprop

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# Directory for Images
# Wira
root_path = "./drive/My Drive/FP_ISIC_Melanoma_Dataset/"
data_training_path = root_path + "ISIC_2019_Training_Input/"
data_testing_path = root_path + "ISIC_2019_Test_Input/"
data_training_label_path = root_path + "ISIC_2019_Training_GroundTruth.csv"
data_labeled_dir = root_path + "labeled_images/"

In [ ]:
# One Time Running
# data_image_label = pd.read_csv(data_training_label_path)
# try:
#   os.mkdir(data_labeled_dir)
# except FileExistsError:
#   pass
# for i in data_image_label.columns:
#   try:
#     os.makedirs(data_labeled_dir+"/"+i)
#   except FileExistsError:
#     pass

In [6]:
# Metadata
data_image_label = pd.read_csv(data_training_label_path)
data_image_label

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Run One Time for processing Image
# data_label = data_image_label["image"][:]

# for i,data in enumerate(data_label):
#   image = data+".jpg"
  # print(image in os.listdir())
  # value_VASC = "VASC" if data_image_label.iloc[i,:]["VASC"] == 1.0 else False
  # value_MEL = "MEL" if data_image_label.iloc[i,:]["MEL"] == 1.0 else False
  # value_SCC = "SCC" if data_image_label.iloc[i,:]["SCC"] == 1.0 else False
  # value_NV = "NV" if data_image_label.iloc[i,:]["NV"] == 1.0 else False
  # value_DF = "DF" if data_image_label.iloc[i,:]["DF"] == 1.0 else False
  # value_BKL = "BKL" if data_image_label.iloc[i,:]["BKL"] == 1.0 else False
  # value_BCC = "BCC" if data_image_label.iloc[i,:]["BCC"] == 1.0 else False
  # value_AK = "AK" if data_image_label.iloc[i,:]["AK"] == 1.0 else False
  # images = [value_VASC,value_MEL,value_SCC,value_NV,value_DF,value_BKL,value_BCC,value_AK]
  # for i in images:
  #   if(i):
  #     label = i
  #     shutil.copy2(data_training_path+image,data_labeled_dir+label)

  # print("{}: {}".format(str(image),str(label)))

In [9]:
# Explore Data
# os.listdir(data_labeled_dir+"MEL")
totalImage = len(os.listdir(data_labeled_dir+"MEL"))+len(os.listdir(data_labeled_dir+"BCC"))+len(os.listdir(data_labeled_dir+"NV"))+len(os.listdir(data_labeled_dir+"AK"))+len(os.listdir(data_labeled_dir+"VASC"))+len(os.listdir(data_labeled_dir+"BKL"))+len(os.listdir(data_labeled_dir+"SCC"))
print("Total Image for BCC: "+str(len(os.listdir(data_labeled_dir+"BCC"))))
print("Total Image for NV: "+str((len(os.listdir(data_labeled_dir+"NV")))))
print("Total Image for AK: "+str((len(os.listdir(data_labeled_dir+"AK")))))
print("Total Image for VASC: "+str((len(os.listdir(data_labeled_dir+"VASC")))))
print("Total Image for BKL: "+str((len(os.listdir(data_labeled_dir+"BKL")))))
print("Total Image for SCC: "+str((len(os.listdir(data_labeled_dir+"SCC")))))
print("Total Image for DF: "+str((len(os.listdir(data_labeled_dir+"DF")))))
print("Total Image for MEL: "+str((len(os.listdir(data_labeled_dir+"MEL")))))

# Total Image
print("Total Image: "+str(totalImage))

Total Image for BCC: 1361
Total Image for NV: 9944
Total Image for AK: 351
Total Image for VASC: 173
Total Image for BKL: 1817
Total Image for SCC: 337
Total Image for DF: 155
Total Image for MEL: 2574
Total Image: 16557


In [10]:
# Import all the necessary files!
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd

In [11]:
# Define a Callback class that stops training once accuracy reaches 97.0%
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.97):
            print("\nReached 97.0% accuracy so cancelling training!")
            self.model.stop_training = True

In [13]:
# Model Hyperparameter
train_data_dir = data_labeled_dir
BATCH_SIZE= 250
#Default 150
WIDTH_SIZE = 148
HEIGHT_SIZE = 148

In [15]:
# WIRA
# train_datagen = ImageDataGenerator(rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     rotation_range=20,
#     horizontal_flip=True,
#     validation_split=0.6) # set validation split

# Training_Input = training (80%), testing(20%) #fokus
# Test_Input = eval

# INDRA
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.5
    )
#

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(WIDTH_SIZE, HEIGHT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training') # set as training data


# WIRA
validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(WIDTH_SIZE, HEIGHT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 8359 images belonging to 8 classes.
Found 8353 images belonging to 8 classes.


In [16]:
# Define the model
callbacks = myCallback()
# WIRA
# Use no more than 2 Conv2D and 2 MaxPooling2D
model = tf.keras.models.Sequential([
    # Your Code Here
    # This is the first convolution
    tf.keras.layers.Conv2D(64,(3,3), activation='relu', input_shape=(WIDTH_SIZE, HEIGHT_SIZE,3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')]
    )

# Compile Model. 
model.compile(loss = 'categorical_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'])

# Compile Model. 
# model.compile(loss = 'categorical_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['acc'])
# model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model
# history = model.fit_generator(train_generator,epochs = 3,validation_data = validation_generator)

In [ ]:
# WIRA
# history = model.fit(train_generator,epochs = 1)

# INDRA
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 100,
            validation_steps = 3,
            verbose = 2)

Epoch 1/100


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["accuracy"])
plt.title("Accuracy")
plt.ylabel('accuracy')
plt.xlabel('epochs')

NameError: ignored

In [ ]:
# model = tf.keras.models.Sequential()
# model.save(root_path+"model_fp.h5")

In [ ]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model(root_path+'model_fp.h5')
# summarize model.
# model.summary()

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 72, 72, 32)        18464     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 9248)             

In [ ]:
os.listdir(root_path)

['ISIC_2019_Training_Input.zip',
 'ISIC_2019_Training_GroundTruth.csv',
 'ISIC_2019_Test_Input.zip',
 'ISIC_2019_Test_Metadata.csv',
 'ISIC_2019_Test_Input',
 'ISIC_2019_Training_Input',
 'ISIC_2019_Training_Metadata.csv',
 'ISIC_2019_Training_Metadata.gsheet',
 'labeled_images',
 'model_history.h5',
 'model_fp.h5',
 'model_fp_72.tflite']

In [ ]:
data_testing_path_image = root_path + "ISIC_2019_Test_Input/ISIC_0071804.jpg"
# os.listdir(data_testing_path)

In [ ]:
data_testing_path_image

'./drive/My Drive/FP_ISIC_Melanoma_Dataset/ISIC_2019_Test_Input/ISIC_0071804.jpg'

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
 
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(128, 128))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 128, 128, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img

image = load_image(data_testing_path_image)

In [ ]:
# model = loa

In [ ]:
model.predict(image)

array([[0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
# Converting a tf.Keras model to a TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
tflitemodel = converter.convert()

In [ ]:
with open("model_fp_72.tflite","wb") as f:
  f.write(tflitemodel)

In [ ]:
!ls

drive	   model_fp_72.tflite	      model_fp.tflite	sample_data
model1.h5  model_fp_converted.tflite  model_history.h5


In [ ]:
shutil.copy2("model_fp_72.tflite",root_path)

'./drive/My Drive/FP_ISIC_Melanoma_Dataset/model_fp_72.tflite'